In [ ]:
!unzip --qq ../input/dogs-vs-cats/train.zip -d .
!unzip -qq ../input/dogs-vs-cats/test1.zip -d .

# Library 

In [ ]:
import torchvision
import torch.nn as nn
import torchvision.transforms as v_transforms
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import glob
import torch
import os
import matplotlib.pyplot as plt

# Data

In [ ]:
all_training_files = glob.glob("./train/*.jpg")[:1000]

In [ ]:
os_training_files = os.listdir("./train")[:1000]

In [ ]:
os_training_files[0]

In [ ]:
len(all_training_files)

In [ ]:
data_df = pd.DataFrame()
data_df['path'] = all_training_files

In [ ]:
data_df.sample(10)

In [ ]:
# manipulate str, path

In [ ]:
os.path.basename(all_training_files[1]).split('.')[0]

In [ ]:
data_df['data_class'] = data_df['path'].apply(lambda x: os.path.basename(x).split('.')[0])

In [ ]:
data_df['encoded_class'] = data_df['data_class'].map({"dog": 0, "cat": 1})

In [ ]:
data_df

In [ ]:
import numpy as np

In [ ]:
image = Image.open(np.random.choice(all_training_files))
print(image.size)
image

# Dataset

In [ ]:
transform = v_transforms.Compose(
    [v_transforms.Resize((256, 256)),
     v_transforms.ToTensor(),
#      v_transforms.Normalize((0.5), (0.5))
    ])

batch_size = 8

In [ ]:
ind = 1

In [ ]:
print(data_df.iloc[ind]['encoded_class']), 
Image.open(data_df.iloc[ind]['path'])

In [ ]:
class CatDogDataset(object):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform
        
        
    def __len__(self):
        return data_df.shape[0]
    
    def __getitem__(self, idx):
        image_path = data_df.iloc[idx]['path']
        label = data_df.iloc[idx]['encoded_class']
        image = Image.open(image_path)
        if transform is not None:
            image = transform(image)
            
        return image, label
            
        

In [ ]:
cat_dog_dataset = CatDogDataset(data_df=data_df, transform=transform)

In [ ]:
len(cat_dog_dataset), data_df.shape[0]

In [ ]:
image, label = cat_dog_dataset[0]

plt.imshow(image.numpy().transpose(1, 2, 0))
plt.axis("off")
plt.title(f"Class: {label}")
plt.show()

In [ ]:
image.shape, label

In [ ]:
train_data_loader = DataLoader(cat_dog_dataset, batch_size=16)

In [ ]:
help(DataLoader)

In [ ]:
for batch in train_data_loader:
    break

In [ ]:
images, labels = batch

In [ ]:
def display_images(images, labels): 

    f, ax = plt.subplots(4,4, figsize=(10,15))

    for i, image_id in enumerate(images[:16]):
        image = images[i]
        label = labels[i]
        ax[i//4, i%4].imshow(image.numpy().transpose(1, 2, 0)) 
        ax[i//4, i%4].axis('off')

        ax[i//4, i%4].set_title(f"Class: {label}")
    plt.tight_layout()
    plt.show() 

In [ ]:
display_images(images, labels)

# Pretrained model

## torchvision

In [ ]:
!pip install --quiet torchsummary

In [ ]:
import torchvision
import torchsummary

In [ ]:
alexnet_model = torchvision.models.alexnet(pretrained=False)

In [ ]:
torchsummary.summary(alexnet_model, input_size=(3, 224, 224), device="cpu")

In [ ]:
alexnet_model

In [ ]:
alexnet_model.features

In [ ]:
class SamplePretrainedModel(nn.Module):
    def __init__(self, num_classes):
        super(SamplePretrainedModel, self).__init__()
        self.num_classes = num_classes
        self.base_model = torchvision.models.alexnet(pretrained=True).features
        self.fc = nn.Linear(12544, num_classes)

    def forward(self, x):
        x = self.base_model(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [ ]:
sample_pretrained_model = SamplePretrainedModel(num_classes=2)

In [ ]:
sample_pretrained_model(torch.rand(1, 3, 256, 256)).shape

In [ ]:
sample_pretrained_model

In [ ]:
!pip install timm

## [timm](https://fastai.github.io/timmdocs/) library

In [ ]:
import timm

In [ ]:
timm_model = timm.create_model('resnet18', num_classes=2)

In [ ]:
torchsummary.summary(timm_model, input_size=(3, 224, 224), device="cpu")